In [40]:
import numpy as np
import pandas as pd
import os, json, math

os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [41]:
import torch

In [42]:
from transformers import AutoModelForSequenceClassification

In [43]:
import pandas as pd
import json

def get_word_index(sentences) :
    all_tokens = []
    for txt in sentences :
        all_tokens += txt.split()

    tokens = pd.Series(all_tokens, range(len(all_tokens)), name="tokens")
    types = tokens.unique()
    word_index = {word : i for i, word in enumerate(["<pad>", "<unk>"] + list(types))}
    
    def decode_review(text):
        nonlocal word_index
        reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
        return ' '.join([reverse_word_index.get(i, '?') for i in text])
    
    return word_index, decode_review

def get_dataset(dataframe, word_index, max_len=256) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    x_series, y_series = dataframe["text"], dataframe["class"]

    x_list = [txt.split() for txt in list(x_series)]
    
    x_seq = []
    for tknlst in x_list :
        seq = []
        for tkn in tknlst :
            try :
                seq.append(word_index[tkn])
            except KeyError :
                seq.append(word_index["<unk>"])
        seq = (seq + [0] * (max_len - len(seq))) if (len(seq) < max_len) else (seq[ : max_len])
        x_seq.append(seq)
    
    # y_int = [1 if lb >= .5 else 0 for lb in list(y_series)]

    x = np.array(x_seq, dtype=int) 
    mask = x != 0

    y = np.array(list(y_series), dtype=float) 

    return x, mask, y

In [44]:
# Para dataset simples, sem folding
# Args
resource_folder = "../resources"
dataset_folder = "StanfordSentimentTreebank"
dataset_name   = "SST2Processed2"
dataset_split   = "test"
experiment_name = f"{dataset_name}-train-hard"
n_classes = 1
batch_size = 16

# Exec
embeddings_df = pd.read_csv(f"{resource_folder}/embeddings/{dataset_folder}/{dataset_name}-train_dim768.csv",        index_col=0)
dataset_df    = pd.read_csv(f"{resource_folder}/datasets/{dataset_folder}/split/{dataset_name}-{dataset_split}.csv", index_col=0)

output_dir    = f"../resources/output/{experiment_name}-Eval"

word_index, decode_review = get_word_index(dataset_df["text"])
x_test, mask_test, y_test = get_dataset(dataset_df, word_index)
print(x_test.shape, mask_test.shape, y_test.shape)
dataset = []
for x, msk, y in zip(x_test, mask_test, y_test) : 
    x   = torch.tensor([int(tkn) for tkn in x])
    msk = torch.tensor([int(tkn) for tkn in msk])
    y   = torch.tensor(float(y))
    dataset.append({'input_ids'      : x,
                    'attention_mask' : msk,
                    'labels'         : y}) 
print(len(dataset))
print(dataset[0])

(2210, 256) (2210, 256) (2210,)
2210
{'input_ids': tensor([2, 3, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
model = AutoModelForSequenceClassification.from_pretrained(
    f"{resource_folder}/output/{experiment_name}/final",
    num_labels=n_classes
)

In [46]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(16177, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [47]:
predictions = []
truths = []
n_batches = math.ceil(len(dataset) / batch_size)
for i_batch in range(n_batches) :
    print("Batch", i_batch + 1, "of", n_batches)
    batch = dataset[i_batch * batch_size : (i_batch + 1) * batch_size]
    
    input_dict = {
        'input_ids'      : torch.tensor([list(sentence['input_ids']) for sentence in batch]), 
        'attention_mask' : torch.tensor([list(sentence['attention_mask']) for sentence in batch]),
        'labels' : torch.tensor([sentence['labels'] for sentence in batch])
    }
    
    preds = model.forward(input_ids      = input_dict['input_ids'], 
                          attention_mask = input_dict['attention_mask'])
    predictions  += [float(logit[0]) for logit in preds['logits']]
    truths       += input_dict['labels']
prediction_column = pd.Series(predictions, dataset_df.index, name="predictions")

Batch 1 of 139
Batch 2 of 139
Batch 3 of 139
Batch 4 of 139
Batch 5 of 139
Batch 6 of 139
Batch 7 of 139
Batch 8 of 139
Batch 9 of 139
Batch 10 of 139
Batch 11 of 139
Batch 12 of 139
Batch 13 of 139
Batch 14 of 139
Batch 15 of 139
Batch 16 of 139
Batch 17 of 139
Batch 18 of 139
Batch 19 of 139
Batch 20 of 139
Batch 21 of 139
Batch 22 of 139
Batch 23 of 139
Batch 24 of 139
Batch 25 of 139
Batch 26 of 139
Batch 27 of 139
Batch 28 of 139
Batch 29 of 139
Batch 30 of 139
Batch 31 of 139
Batch 32 of 139
Batch 33 of 139
Batch 34 of 139
Batch 35 of 139
Batch 36 of 139
Batch 37 of 139
Batch 38 of 139
Batch 39 of 139
Batch 40 of 139
Batch 41 of 139
Batch 42 of 139
Batch 43 of 139
Batch 44 of 139
Batch 45 of 139
Batch 46 of 139
Batch 47 of 139
Batch 48 of 139
Batch 49 of 139
Batch 50 of 139
Batch 51 of 139
Batch 52 of 139
Batch 53 of 139
Batch 54 of 139
Batch 55 of 139
Batch 56 of 139
Batch 57 of 139
Batch 58 of 139
Batch 59 of 139
Batch 60 of 139
Batch 61 of 139
Batch 62 of 139
Batch 63 of 139
B

In [48]:
dataset_with_preds_df = pd.concat([dataset_df, prediction_column], axis=1)

In [49]:
if not os.path.exists(output_dir) :
    os.makedirs(output_dir)
dataset_with_preds_df.to_csv(f"{output_dir}/Predictions.csv")

In [50]:
dataset_with_preds_df

,text,phrase_id,class,predictions
3,<start> effective but too tepid biopic,13995,0.513890,0.813343
4,<start> if you sometimes like to go to the mov...,14123,0.736110,0.729453
5,<start> emerges as something rare an issue mov...,13999,0.861110,0.898561
6,<start> the film provides some great insight i...,14498,0.597220,0.874723
7,<start> offers that rare combination of entert...,14351,0.833330,0.904525
...,...,...,...,...
11621,<start> an imaginative comedythriller,13851,0.777780,0.530839
11623,<start> a rare beautiful film,18182,0.916670,0.939123
11626,<start> an hilarious romantic comedy,23211,0.888890,0.948151
11628,<start> never sinks into exploitation,26177,0.625000,0.731718


In [51]:
hits, total = 0, 0
for _, row in dataset_with_preds_df.iterrows() :
    total += 1
    pred  = row["predictions"] >= .5
    label = row["class"]       >= .5
    if pred == label : 
        hits += 1
accuracy = hits / total
accuracy

0.5579185520361991

In [52]:
hits, total = 0, 0
for _, row in dataset_with_preds_df.iterrows() :
    total += 1
    pred  = row["predictions"]
    label = row["class"]
    diff = label - pred
    hits += abs(diff)
avg_dev = hits / total
avg_dev

0.32979248465010524

In [53]:
true_positives, true_negatives, false_positives, false_negatives = 0, 0, 0, 0
for _, row in dataset_with_preds_df.iterrows() :
    pred  = row["predictions"] >= .5
    label = row["class"]       >= .5
    if pred == label :
        if pred :
            true_positives  += 1
        else :
            true_negatives  += 1
    else :
        if pred :
            false_positives += 1
        else :
            false_negatives += 1
print(true_positives, true_negatives, false_positives, false_negatives)

921 312 787 190


In [54]:
precision = true_positives / (true_positives + false_positives)
precision

0.5392271662763466

In [55]:
recall = true_positives / (true_positives + false_negatives)
recall

0.828982898289829

In [56]:
f1 = (2 * precision * recall) / (precision + recall)
f1

0.6534231997162114